# Import necessary modules and load sql engine

In [377]:
import sqlalchemy
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sqlalchemy import create_engine
from IPython.display import display_html
from datetime import datetime
from pytz import timezone

In [378]:
%load_ext sql
%sql postgresql://postgres@localhost/homedata
engine = create_engine('postgresql://postgres@localhost/homedata')

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


# Declare configuration variables

In [417]:
# constant variables
fc = {'work':'black', 
      'eating':'green', 
      'toilet':'purple', 
      'fitness':'red', 
      'sleep':'skyblue', 
      'personal_hygiene':'grey', 
      'shower':'blue', 
      'relax':'orange', 
      'cooking':'pink', 
      'phonecall':'teal', 
      'leave_home':'brown'}

# visualization targets and period
target_period = '168 hours'

# Fetch sensor data from TimescaleDB

In [418]:
# MTS Sensor data fetch
hroom1_result = %sql SELECT * FROM sensor_data WHERE db_insert_time > now() - interval '{target_period}' AND room = 'hroom-1'
hroom2_result = %sql SELECT * FROM sensor_data WHERE db_insert_time > now() - interval '{target_period}' AND room = 'hroom-2'
act_result = %sql SELECT data_gen_time, activity, start_end FROM user_activity WHERE data_gen_time > now() - interval '{target_period}'

sensor_df1 = hroom1_result.DataFrame()
sensor_df2 = hroom2_result.DataFrame()
act_df = act_result.DataFrame()

sensor_df1.sort_values('db_insert_time', ascending = True, inplace = True)
sensor_df2.sort_values('db_insert_time', ascending = True, inplace = True)
act_df.sort_values('data_gen_time', ascending = True, inplace = True)

# Avg value fetch for sensor data
r1s1 = %sql SELECT db_insert_time, (select avg(a) from unnest(sound1) as a) FROM sensor_data WHERE db_insert_time > now() - interval '{target_period}' AND room = 'hroom-1'
r1s2 = %sql SELECT db_insert_time, (select avg(a) from unnest(sound2) as a) FROM sensor_data WHERE db_insert_time > now() - interval '{target_period}' AND room = 'hroom-1'
r2s1 = %sql SELECT db_insert_time, (select avg(a) from unnest(sound1) as a) FROM sensor_data WHERE db_insert_time > now() - interval '{target_period}' AND room = 'hroom-2'
r2s2 = %sql SELECT db_insert_time, (select avg(a) from unnest(sound2) as a) FROM sensor_data WHERE db_insert_time > now() - interval '{target_period}' AND room = 'hroom-2'

r1s1_df = r1s1.DataFrame()
r1s2_df = r1s2.DataFrame()
r2s1_df = r2s1.DataFrame()
r2s2_df = r2s2.DataFrame()

r1s1_df.sort_values('db_insert_time', ascending = True, inplace = True)
r1s2_df.sort_values('db_insert_time', ascending = True, inplace = True)
r2s1_df.sort_values('db_insert_time', ascending = True, inplace = True)
r2s2_df.sort_values('db_insert_time', ascending = True, inplace = True)

 * postgresql://postgres@localhost/homedata
451586 rows affected.
 * postgresql://postgres@localhost/homedata
450526 rows affected.
 * postgresql://postgres@localhost/homedata
247 rows affected.
 * postgresql://postgres@localhost/homedata
451587 rows affected.
 * postgresql://postgres@localhost/homedata
451587 rows affected.
 * postgresql://postgres@localhost/homedata
450526 rows affected.
 * postgresql://postgres@localhost/homedata
450526 rows affected.


# Merge data into a single dataframe in a normalized scale

## Merge Dataframe along the same timescale

### Time-series friendly merging: pd.merge_asof()

#### Sound Sensor

In [419]:
r1s1_df.rename(columns={'avg':'snd11_roadside'}, inplace=True)
r1s2_df.rename(columns={'avg':'snd12_bed'}, inplace=True)
r2s1_df.rename(columns={'avg':'snd21_kitchen'}, inplace=True)
r2s2_df.rename(columns={'avg':'snd22_desk'}, inplace=True)

r1_df = pd.merge_asof(r1s1_df, r1s2_df, on='db_insert_time', tolerance=pd.Timedelta('5ms'))
r2_df = pd.merge_asof(r2s1_df, r2s2_df, on='db_insert_time', tolerance=pd.Timedelta('5ms'))

snd_df = pd.merge_asof(r1_df, r2_df, on='db_insert_time', tolerance=pd.Timedelta('1s'))
snd_df.rename(columns={'db_insert_time':'time'}, inplace=True)
snd_df.dropna(inplace=True)

#### Other Sensors

In [420]:
sensor_df1.drop(columns=['data_gen_time', 'room', 'sound1', 'sound2', 'airquality', 'dustconcentration'], inplace=True)
sensor_df2.drop(columns=['data_gen_time', 'room', 'sound1', 'sound2', 'airquality', 'dustconcentration'], inplace=True)

sensor_df1.rename(columns={'db_insert_time':'time', 'temperature':'tmp1', 'humidity':'hmd1', 'light':'light1', 'motion':'mot1'}, inplace=True)
sensor_df2.rename(columns={'db_insert_time':'time', 'temperature':'tmp2', 'humidity':'hmd2', 'light':'light2', 'motion':'mot2'}, inplace=True)

sensor_df = pd.merge_asof(snd_df, sensor_df1, on='time', tolerance=pd.Timedelta('1s'))
sensor_df = pd.merge_asof(sensor_df, sensor_df2, on='time', tolerance=pd.Timedelta('1s'))
sensor_df.dropna(inplace=True)

#### Addition of Weekday / Weekend

In [421]:
# add weekdays / weekend info into the DataFrame
# weekno = sensor_df['time'][0].to_pydatetime().weekday() # if weekno <5 ? weekday : weekend
sensor_df['weekend'] = sensor_df.apply(lambda x: int(x['time'].to_pydatetime().weekday()/5), axis=1)

# add hour info into the DataFrame
sensor_df['hour'] = sensor_df.apply(lambda x: float(x['time'].to_pydatetime().hour/24), axis=1)

#### Synchronize time scale of activity dataframe to sensor dataframe

In [52]:
# drop activity rows earlier than the head of sensor_df

# drop activity rows later than the tail of sensor_df


In [422]:
# initialize plotting
ax = plt.gca()

# add sensor values to the target plot
sensor_df.plot(kind='line', x='time', y='snd11_roadside', ax=ax)
sensor_df.plot(kind='line', x='time', y='snd12_bed', ax=ax)
sensor_df.plot(kind='line', x='time', y='snd21_kitchen', ax=ax)
sensor_df.plot(kind='line', x='time', y='snd22_desk', ax=ax)
sensor_df.plot(kind='line', x='time', y='tmp1', ax=ax)
sensor_df.plot(kind='line', x='time', y='tmp2', ax=ax)
sensor_df.plot(kind='line', x='time', y='hmd1', ax=ax)
sensor_df.plot(kind='line', x='time', y='hmd2', ax=ax)
sensor_df.plot(kind='line', x='time', y='mot1', ax=ax)
sensor_df.plot(kind='line', x='time', y='mot2', ax=ax)
sensor_df.plot(kind='line', x='time', y='light1', ax=ax)
sensor_df.plot(kind='line', x='time', y='light2', ax=ax)
sensor_df.plot(kind='line', x='time', y='hour', ax=ax)

# add activity periods to the target plot
act_stack = {} # stack implemented as a dict which stores unclosed/unfinished activities to be visualized

start_T = ''
end_T = ''

log = {'err':'', 'debug':''}

# converts numpy.datetime64 to datetime.
# datetime is compatible with pandas.timestamp with timezone.
def convT(T_dt64):
    T_ts = (T_dt64 - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    return datetime.fromtimestamp(T_ts).astimezone(timezone('US/Eastern'))

# visualize starts here
for index, row in act_df.iterrows():    
    act = row['activity']
    
    if row['start_end'] == 'start':
        if act in act_stack.keys():
            log['err'] += 'The given activity is already in the activity stack.' # error message
        else:
            act_stack[act] = {}
            log['debug'] += '{} is added to act_stack.\n'.format(act)
            #print('{} is added to act_stack.'.format(act))

            start_T = row['data_gen_time']

            if index == act_df.size-1: # if this is the last row
                end_T = convT(sensor_df.tail(1)['time'].values[0])
                log['debug'] += "act={}, start_T={}, end_T={}, color={}\n".format(act, str(start_T), str(end_T), fc[act])
                ax.axvspan(start_T, end_T, label = act, facecolor=fc[act], alpha=0.2)
                act_stack.pop(act)
                log['debug'] += '[1] act_stack.pop({})\n'.format(act)
            else:
                act_stack[act]['start_T'] = start_T
    else: # row['start_end'] == 'end'
        end_T = row['data_gen_time']
        
        if act in act_stack.keys():
            act_stack[act]['end_T'] = end_T
            log['debug'] += "act={}, start_T={}, end_T={}, color={}\n".format(act, str(act_stack[act]['start_T']), str(act_stack[act]['end_T']), fc[act])
            ax.axvspan(act_stack[act]['start_T'], act_stack[act]['end_T'], label = act, facecolor=fc[act], alpha=0.2)
            act_stack.pop(act)
            log['debug'] += '[2] act_stack.pop({})\n'.format(act)
        else:
            if start_T == '':
                start_T = convT(sensor_df.head(1)['time'].values[0])
            
            log['debug'] += "act={}, start_T={}, end_T={}, color={}\n".format(act, str(start_T), str(end_T), fc[act])
            ax.axvspan(start_T, end_T, label = act, facecolor=fc[act], alpha=0.2)
            
if len(act_stack.keys()) != 0:
    log['debug'] += "len(act_stack.keys()) = {}\n".format(len(act_stack.keys()))
    log['debug'] += "act_stack = {}\n".format(str(act_stack))
    
    for act in act_stack.keys():
        end_T = convT(sensor_df.tail(1)['time'].values[0])
        ax.axvspan(start_T, end_T, label = act, facecolor=fc[act], alpha=0.2)


# plot
plt.rcParams["figure.figsize"] = (20,10)
plt.show()

/Users/heesuk/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


## Normalize Dataframe values along all columns
To facilitate any machine learning training phases, the sensor values will be normalized into a range [0, 1], considering the possible data range of each sensor.

#### Valid data range of each used sensors:
* Sound Phidget: 34~102 dB (SPL)
* Humidity/Temperature Phidget: -30 ~ 80C, 10 ~ 95%
* Precision Light Phidget: 1000 lux
* Motion Phidget: Not found but maybe 0~10

#### How to normalize each sensor:
* Sound Phidget: (x-30)/50, assuming that sound level never goes beyond 80dB
* Humidity: x/95, since x-10<0 is found
* Temperature: (x+20)/70, assuming that temp. never goes down to -20C either up to 50C
* Light: x/1000
* Motion: x/10

In [423]:
sensor_df['snd11_roadside'] = sensor_df.apply(lambda x: (x['snd11_roadside']-30)/50, axis=1)
sensor_df['snd12_bed'] = sensor_df.apply(lambda x: (x['snd12_bed']-30)/50, axis=1)
sensor_df['snd21_kitchen'] = sensor_df.apply(lambda x: (x['snd21_kitchen']-30)/50, axis=1)
sensor_df['snd22_desk'] = sensor_df.apply(lambda x: (x['snd22_desk']-30)/50, axis=1)
sensor_df['tmp1'] = sensor_df.apply(lambda x: (x['tmp1']+20)/70, axis=1)
sensor_df['tmp2'] = sensor_df.apply(lambda x: (x['tmp2']+20)/70, axis=1)
sensor_df['hmd1'] = sensor_df.apply(lambda x: x['hmd1']/95, axis=1)
sensor_df['hmd2'] = sensor_df.apply(lambda x: x['hmd2']/95, axis=1)
sensor_df['light1'] = sensor_df.apply(lambda x: x['light1']/1000, axis=1)
sensor_df['light2'] = sensor_df.apply(lambda x: x['light2']/1000, axis=1)
sensor_df['mot1'] = sensor_df.apply(lambda x: x['mot1']/10, axis=1)
sensor_df['mot2'] = sensor_df.apply(lambda x: x['mot2']/10, axis=1)
sensor_df

time  snd11_roadside  snd12_bed  \
0      2020-05-07 14:55:35.605686-04:00        0.116803   0.085431   
1      2020-05-07 14:55:37.854700-04:00        0.131102   0.083911   
2      2020-05-07 14:55:40.099457-04:00        0.132529   0.091723   
3      2020-05-07 14:55:41.220124-04:00        0.128318   0.097108   
4      2020-05-07 14:55:42.334265-04:00        0.161665   0.101104   
...                                 ...             ...        ...   
236782 2020-05-14 14:54:52.693961-04:00        0.123358   0.092959   
236783 2020-05-14 14:54:56.299927-04:00        0.133627   0.085450   
236784 2020-05-14 14:55:02.026793-04:00        0.123699   0.087165   
236785 2020-05-14 14:55:03.141959-04:00        0.128148   0.094960   
236786 2020-05-14 14:55:04.254029-04:00        0.130400   0.088708   

        snd21_kitchen  snd22_desk      tmp1      hmd1   light1      mot1  \
0            0.130023    0.136501  0.569157  0.219484  0.25723  0.248077   
1            0.141076    0.140690  0.569257  0.219368  0.25769  0.252060   
2            0.134810    0.137813  0.569057  0.219179  0.25656  0.250641   
3            0.139950    0.150062  0.569057  0.219547  0.26065  0.244583   
4            0.151034    0.156616  0.569157  0.219368  0.26093  0.246704   
...               ...         ...       ...       ...      ...       ...   
236782       0.136820    0.127161  0.574871  0.159495  0.19467  0.240250   
236783       0.134290    0.132207  0.575057  0.159853  0.19061  0.248947   
236784       0.143468    0.118901  0.575057  0.159558  0.18671  0.247971   
236785       0.141689    0.131183  0.574971  0.159421  0.19394  0.248871   
236786       0.139868    0.126162  0.574971  0.159495  0.18808  0.245819   

            tmp2      hmd2   light2      mot2  weekend      hour  
0       0.576143  0.224758  0.29742  0.244629        0  0.583333  
1       0.576229  0.225000  0.29855  0.245697        0  0.583333  
2       0.576229  0.224642  0.29672  0.236282        0  0.583333  
3       0.576143  0.224642  0.29843  0.242523        0  0.583333  
4       0.576143  0.224758  0.29861  0.269440        0  0.583333  
...          ...       ...      ...       ...      ...       ...  
236782  0.580871  0.166337  0.22070  0.181702        0  0.583333  
236783  0.580671  0.166600  0.22256  0.231781        0  0.583333  
236784  0.580771  0.166779  0.22003  0.262024        0  0.583333  
236785  0.580671  0.166716  0.22281  0.252792        0  0.583333  
236786  0.580871  0.166274  0.21921  0.200668        0  0.583333  

[236787 rows x 15 columns]

#### Validate the data in range [0,1]

In [401]:
sensor_df[(sensor_df[['snd11_roadside', 'snd12_bed', 'snd21_kitchen', 'snd22_desk', 'tmp1', 'hmd1', 'light1', 'mot1', 'tmp2', 'hmd2', 'light2', 'mot2', 'weekend', 'hour']] > 1).any(1)]
#(sensor_df[['snd11_roadside', 'snd12_bed', 'snd21_kitchen', 'snd22_desk', 'tmp1', 'hmd1', 'light1', 'mot1', 'tmp2', 'hmd2', 'light2', 'mot2', 'weekend', 'hour']] > 0).all(1)

Empty DataFrame
Columns: [time, snd11_roadside, snd12_bed, snd21_kitchen, snd22_desk, tmp1, hmd1, light1, mot1, tmp2, hmd2, light2, mot2, weekend, hour]
Index: []

# Visualize the MTS data with activity data labels

In [424]:
# initialize plotting
ax = plt.gca()

# add sensor values to the target plot
sensor_df.plot(kind='line', x='time', y='snd11_roadside', ax=ax)
sensor_df.plot(kind='line', x='time', y='snd12_bed', ax=ax)
sensor_df.plot(kind='line', x='time', y='snd21_kitchen', ax=ax)
sensor_df.plot(kind='line', x='time', y='snd22_desk', ax=ax)
sensor_df.plot(kind='line', x='time', y='tmp1', ax=ax)
sensor_df.plot(kind='line', x='time', y='tmp2', ax=ax)
sensor_df.plot(kind='line', x='time', y='hmd1', ax=ax)
sensor_df.plot(kind='line', x='time', y='hmd2', ax=ax)
sensor_df.plot(kind='line', x='time', y='mot1', ax=ax)
sensor_df.plot(kind='line', x='time', y='mot2', ax=ax)
sensor_df.plot(kind='line', x='time', y='light1', ax=ax)
sensor_df.plot(kind='line', x='time', y='light2', ax=ax)
sensor_df.plot(kind='line', x='time', y='weekend', ax=ax)
sensor_df.plot(kind='line', x='time', y='hour', ax=ax)

# add activity periods to the target plot
act_stack = {} # stack implemented as a dict which stores unclosed/unfinished activities to be visualized

start_T = ''
end_T = ''

log = {'err':'', 'debug':''}

# converts numpy.datetime64 to datetime.
# datetime is compatible with pandas.timestamp with timezone.
def convT(T_dt64):
    T_ts = (T_dt64 - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')
    return datetime.fromtimestamp(T_ts).astimezone(timezone('US/Eastern'))

# visualize starts here
for index, row in act_df.iterrows():    
    act = row['activity']
    
    if row['start_end'] == 'start':
        if act in act_stack.keys():
            log['err'] += 'The given activity is already in the activity stack.' # error message
        else:
            act_stack[act] = {}
            log['debug'] += '{} is added to act_stack.\n'.format(act)
            #print('{} is added to act_stack.'.format(act))

            start_T = row['data_gen_time']

            if index == act_df.size-1: # if this is the last row
                end_T = convT(sensor_df.tail(1)['time'].values[0])
                log['debug'] += "act={}, start_T={}, end_T={}, color={}\n".format(act, str(start_T), str(end_T), fc[act])
                ax.axvspan(start_T, end_T, label = act, facecolor=fc[act], alpha=0.2)
                act_stack.pop(act)
                log['debug'] += '[1] act_stack.pop({})\n'.format(act)
            else:
                act_stack[act]['start_T'] = start_T
    else: # row['start_end'] == 'end'
        end_T = row['data_gen_time']
        
        if act in act_stack.keys():
            act_stack[act]['end_T'] = end_T
            log['debug'] += "act={}, start_T={}, end_T={}, color={}\n".format(act, str(act_stack[act]['start_T']), str(act_stack[act]['end_T']), fc[act])
            ax.axvspan(act_stack[act]['start_T'], act_stack[act]['end_T'], label = act, facecolor=fc[act], alpha=0.2)
            act_stack.pop(act)
            log['debug'] += '[2] act_stack.pop({})\n'.format(act)
        else:
            if start_T == '':
                start_T = convT(sensor_df.head(1)['time'].values[0])
            
            log['debug'] += "act={}, start_T={}, end_T={}, color={}\n".format(act, str(start_T), str(end_T), fc[act])
            ax.axvspan(start_T, end_T, label = act, facecolor=fc[act], alpha=0.2)
            
if len(act_stack.keys()) != 0:
    log['debug'] += "len(act_stack.keys()) = {}\n".format(len(act_stack.keys()))
    log['debug'] += "act_stack = {}\n".format(str(act_stack))
    
    for act in act_stack.keys():
        end_T = convT(sensor_df.tail(1)['time'].values[0])
        ax.axvspan(start_T, end_T, label = act, facecolor=fc[act], alpha=0.2)


# plot
plt.rcParams["figure.figsize"] = (20,10)
plt.show()

/Users/heesuk/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future


### Color Mapping to Activities
* Work = Black
* Eating = Green
* Toilet = Purple
* Fitness = Red
* Sleep = Skyblue
* Personal_hygiene = Grey
* Shower = Blue
* Relax = Orange
* Cooking = Pink
* Phonecall = Teal

# Construct (X,Y) pairs out of the dataframe for DNN models

## Transformation from dataframe to (X,Y) pairs

### Label each tuple of sensor data with an activity annotation

In [425]:
NULL = -1
NORM_ACT_SESS = 0
OPEN_END_SESS = 1
ONGOING_SESS = 2

act_map = {'work':0, 
           'eating':1, 
           'toilet':2, 
           'fitness':3, 
           'sleep':4, 
           'personal_hygiene':5, 
           'shower':6, 
           'relax':7, 
           'cooking':8, 
           'phonecall':9, 
           'leave_home':10, 
           'null':11}
act_sess_list = []
act_session = {'act':'', 'start':'', 'end':'', 'cls':NULL}

for index, row in act_df.iterrows():
    if row['start_end'] == 'start':
        if index == 0: # open-ended start of an activity
            act_session['act'] = row['activity']
            act_session['start'] = row['data_gen_time']
            act_session['cls'] = OPEN_END_SESS
            act_sess_list.append(act_session.copy())
            act_session = {'act':'', 'start':'', 'end':'', 'cls':NULL}
        else: # normal start - start of an activity to be ended
            act_session['act'] = row['activity']
            act_session['start'] = row['data_gen_time']
    else: # row['start_end'] == 'end'
        if act_session['start'] == '': # an activity started but not shown in the dataframe
            act_session['act'] = row['activity']
            act_session['end'] = row['data_gen_time']
            act_session['cls'] = ONGOING_SESS
            act_sess_list.append(act_session.copy())
            act_session = {'act':'', 'start':'', 'end':'', 'cls':NULL}
        else: # normal end - end of an activity whose start was already added to act_session dict
            act_session['end'] = row['data_gen_time']
            act_session['cls'] = NORM_ACT_SESS
            act_sess_list.append(act_session.copy())
            act_session = {'act':'', 'start':'', 'end':'', 'cls':NULL}

def getAct(test_t): # test_t <- TimeStamp
    result_act = 'null'
    
    for act_sess in act_sess_list:
        if act_sess['cls'] == NORM_ACT_SESS:
            if test_t >= act_sess['start'] and test_t < act_sess['end']:
                result_act = act_sess['act']
                return act_map[result_act]
        elif act_sess['cls'] == OPEN_END_SESS:
            if test_t >= act_sess['start']:
                result_act = act_sess['act']
                return act_map[result_act]
        else: # ONGOING SESS
            if test_t < act_sess['end']:
                result_act = act_sess['act']
                return act_map[result_act]
    
    return act_map[result_act]

In [426]:
sensor_df['act'] = sensor_df['time'].apply(getAct)
sensor_df

time  snd11_roadside  snd12_bed  \
0      2020-05-07 14:55:35.605686-04:00        0.116803   0.085431   
1      2020-05-07 14:55:37.854700-04:00        0.131102   0.083911   
2      2020-05-07 14:55:40.099457-04:00        0.132529   0.091723   
3      2020-05-07 14:55:41.220124-04:00        0.128318   0.097108   
4      2020-05-07 14:55:42.334265-04:00        0.161665   0.101104   
...                                 ...             ...        ...   
236782 2020-05-14 14:54:52.693961-04:00        0.123358   0.092959   
236783 2020-05-14 14:54:56.299927-04:00        0.133627   0.085450   
236784 2020-05-14 14:55:02.026793-04:00        0.123699   0.087165   
236785 2020-05-14 14:55:03.141959-04:00        0.128148   0.094960   
236786 2020-05-14 14:55:04.254029-04:00        0.130400   0.088708   

        snd21_kitchen  snd22_desk      tmp1      hmd1   light1      mot1  \
0            0.130023    0.136501  0.569157  0.219484  0.25723  0.248077   
1            0.141076    0.140690  0.569257  0.219368  0.25769  0.252060   
2            0.134810    0.137813  0.569057  0.219179  0.25656  0.250641   
3            0.139950    0.150062  0.569057  0.219547  0.26065  0.244583   
4            0.151034    0.156616  0.569157  0.219368  0.26093  0.246704   
...               ...         ...       ...       ...      ...       ...   
236782       0.136820    0.127161  0.574871  0.159495  0.19467  0.240250   
236783       0.134290    0.132207  0.575057  0.159853  0.19061  0.248947   
236784       0.143468    0.118901  0.575057  0.159558  0.18671  0.247971   
236785       0.141689    0.131183  0.574971  0.159421  0.19394  0.248871   
236786       0.139868    0.126162  0.574971  0.159495  0.18808  0.245819   

            tmp2      hmd2   light2      mot2  weekend      hour  act  
0       0.576143  0.224758  0.29742  0.244629        0  0.583333    0  
1       0.576229  0.225000  0.29855  0.245697        0  0.583333    0  
2       0.576229  0.224642  0.29672  0.236282        0  0.583333    0  
3       0.576143  0.224642  0.29843  0.242523        0  0.583333    0  
4       0.576143  0.224758  0.29861  0.269440        0  0.583333    0  
...          ...       ...      ...       ...      ...       ...  ...  
236782  0.580871  0.166337  0.22070  0.181702        0  0.583333   11  
236783  0.580671  0.166600  0.22256  0.231781        0  0.583333   11  
236784  0.580771  0.166779  0.22003  0.262024        0  0.583333   11  
236785  0.580671  0.166716  0.22281  0.252792        0  0.583333   11  
236786  0.580871  0.166274  0.21921  0.200668        0  0.583333   11  

[236787 rows x 16 columns]

### Transform each DataFrame row into a tuple of (x,y)

In [427]:
subset_x = sensor_df[['snd11_roadside', 'snd12_bed', 'snd21_kitchen', 'snd22_desk', 'tmp1', 'hmd1', 'light1', 'mot1', 'tmp2', 'hmd2', 'light2', 'mot2', 'weekend', 'hour']]
subset_y = sensor_df['act']
home_dset_features = subset_x.values.tolist()
home_dset_labels = subset_y.tolist()

type(home_dset_labels), len(home_dset_labels), type(home_dset_features), len(home_dset_features)

(list, 236787, list, 236787)

# Dataset Preparation for 2D-CNN, including 2D-ResNet
Considering that each time gap between two data points is 1.2s in average, 50 data points are expected to compose one-minute long 3D data. By adding two zero-valued features and reshaping the features, we can compose each datapoint of a 50x4x4 cubic shape. This transformation should be optimal to CNN-like network architectures because it can be used as the input of any CNN-like models as-is. It is also possible to use 50x7x2 cubic shape, not adding two null features with increased number of parameters. However, squared shape should be much more effective to capture correlated patterns of input features because a bigger kernel (e.g., 3x3) can be used.

### Adding null features

In [428]:
sensor_df['null-1'] = 0
sensor_df['null-2'] = 0
sensor_df = sensor_df.reindex(columns = ['time', 'snd11_roadside', 'snd12_bed', 'snd21_kitchen', 'snd22_desk', 'tmp1', 'hmd1', 'light1', 'mot1', 'tmp2', 'hmd2', 'light2', 'mot2', 'weekend', 'hour', 'null-1', 'null-2', 'act'])
sensor_df

time  snd11_roadside  snd12_bed  \
0      2020-05-07 14:55:35.605686-04:00        0.116803   0.085431   
1      2020-05-07 14:55:37.854700-04:00        0.131102   0.083911   
2      2020-05-07 14:55:40.099457-04:00        0.132529   0.091723   
3      2020-05-07 14:55:41.220124-04:00        0.128318   0.097108   
4      2020-05-07 14:55:42.334265-04:00        0.161665   0.101104   
...                                 ...             ...        ...   
236782 2020-05-14 14:54:52.693961-04:00        0.123358   0.092959   
236783 2020-05-14 14:54:56.299927-04:00        0.133627   0.085450   
236784 2020-05-14 14:55:02.026793-04:00        0.123699   0.087165   
236785 2020-05-14 14:55:03.141959-04:00        0.128148   0.094960   
236786 2020-05-14 14:55:04.254029-04:00        0.130400   0.088708   

        snd21_kitchen  snd22_desk      tmp1      hmd1   light1      mot1  \
0            0.130023    0.136501  0.569157  0.219484  0.25723  0.248077   
1            0.141076    0.140690  0.569257  0.219368  0.25769  0.252060   
2            0.134810    0.137813  0.569057  0.219179  0.25656  0.250641   
3            0.139950    0.150062  0.569057  0.219547  0.26065  0.244583   
4            0.151034    0.156616  0.569157  0.219368  0.26093  0.246704   
...               ...         ...       ...       ...      ...       ...   
236782       0.136820    0.127161  0.574871  0.159495  0.19467  0.240250   
236783       0.134290    0.132207  0.575057  0.159853  0.19061  0.248947   
236784       0.143468    0.118901  0.575057  0.159558  0.18671  0.247971   
236785       0.141689    0.131183  0.574971  0.159421  0.19394  0.248871   
236786       0.139868    0.126162  0.574971  0.159495  0.18808  0.245819   

            tmp2      hmd2   light2      mot2  weekend      hour  null-1  \
0       0.576143  0.224758  0.29742  0.244629        0  0.583333       0   
1       0.576229  0.225000  0.29855  0.245697        0  0.583333       0   
2       0.576229  0.224642  0.29672  0.236282        0  0.583333       0   
3       0.576143  0.224642  0.29843  0.242523        0  0.583333       0   
4       0.576143  0.224758  0.29861  0.269440        0  0.583333       0   
...          ...       ...      ...       ...      ...       ...     ...   
236782  0.580871  0.166337  0.22070  0.181702        0  0.583333       0   
236783  0.580671  0.166600  0.22256  0.231781        0  0.583333       0   
236784  0.580771  0.166779  0.22003  0.262024        0  0.583333       0   
236785  0.580671  0.166716  0.22281  0.252792        0  0.583333       0   
236786  0.580871  0.166274  0.21921  0.200668        0  0.583333       0   

        null-2  act  
0            0    0  
1            0    0  
2            0    0  
3            0    0  
4            0    0  
...        ...  ...  
236782       0   11  
236783       0   11  
236784       0   11  
236785       0   11  
236786       0   11  

[236787 rows x 18 columns]

### Export preprocessed data to CSV file

In [430]:
sensor_df.to_csv('act_mts_data_200514_7d.csv')